In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

<img src="https://upload.wikimedia.org/wikipedia/commons/c/c7/HEIG-VD_Logo_96x29_RVB_ROUGE.png" alt="HEIG-VD Logo" width="250"/>

# Cours TAL – Mini-projet
# Classification de dépêches d’agence avec NLTK

> Tiago Povoa Quinteiro

**Modalités du projet**

L’objectif de ce projet est de réaliser des expériences de classification de documents sous NLTK avec
le corpus de dépêches Reuters. Le projet est individuel : vous êtes responsable des différentes options
choisies, et en principe les résultats de chaque projet seront différents. Le projet sera jugé sur la
qualité des expériences (correction méthodologique) mais aussi sur la discussion des options
explorées.

Vous devez remettre un notebook Jupyter présentant vos choix, votre code, vos résultats et les
discussions. Le notebook devra déjà contenir les résultats des exécutions, mais pourra être ré-exécuté
par le professeur en vue d’une vérification.

Vous devrez en outre faire une courte présentation orale (5-7 min.) et répondre aux questions sur
votre projet (5-7 min.) lors d’une séance sur Teams (15 min.) avec le professeur et l’assistant.

**Description des expériences**

1. **L’objectif général** est d’explorer au moins deux aspects parmi les multiples choix qui se posent lors de la création d’un système de classification de textes.
2. **Données** : les dépêches du corpus Reuters, tel qu’il est fourni par NLTK. Vous respecterez notamment la division en données d’entraînement (train) et données de test.
3. **Hyper-paramètres** : la définition d’un classifieur comporte un grand nombre de choix de conception, dans plusieurs dimensions. Dans ce projet, et pour chaque objectif de classification (voir ci-dessous) vous explorerez deux dimensions. Pour chaque dimension, vous comparerez au moins deux options pour trouver laquelle fournit le meilleur score, et vous tenterez d’expliquer pourquoi. Vous pourrez choisir parmi les options suivantes :

    a. options de prétraitement des textes : stopwords, lemmatisation, tout en minuscules.
    
    b. options de représentation : présence/absence de mots indicateurs, nombre de mots indicateurs ; présence/absence/nombre de bigrammes, trigrammes ; autres traits : longueur de la dépêche, rapport tokens/types.
    
    c. classifieurs et leurs paramètres : divers choix possibles (voir la documentation).
    
    
4. **Objectif de classification** : vous devrez construire quatre classifieurs. Vous choisirez les meilleurs hyper-paramètres pour chaque classifieur sans regarder les résultats sur les données de test NLTK, mais en divisant les données d’entraînement NLTK en 80% train et 20% dev. Vous ferez ensuite l’entraînement final sur l’intégralité des données d’entraînement.

    a. Veuillez d’abord définir et entraîner trois classifieurs binaires, correspondant à trois catégories de votre choix. Chaque classifieur prédit si une dépêche appartient ou non à la catégorie, i.e. si elle doit recevoir ou non l’étiquette respective. Veuillez construire un premier classifieur binaire pour une étiquette que vous choisirez librement parmi les trois suivantes : ‘money-fx’, ‘interest’, ou ‘money-supply’. Le deuxième classifieur binaire concernera une étiquette de votre choix parmi : ‘grain’, ‘wheat’, ‘corn’. Enfin, le troisième sera choisi parmi : ‘crude’, ‘nat-gas’, ‘gold’.
        - Veuillez donner les scores de rappel, précision et f-mesure de chacun des trois classifieurs que vous avez conçus et entraînés.
    
    b. On vous demande également de définir un quatrième classifieur qui assigne l’une des trois étiquettes que vous avez choisies ci-dessus plus la catégorie ‘other’ (il assigne donc une seule étiquette parmi quatre). Vous devrez adapter légèrement les données, car un très petit nombre de dépêches (combien ?) sont en réalité annotées avec plusieurs de ces étiquettes, et vous n’en retiendrez que la première.
        - Veuillez évaluer ce classifieur en termes de rappel, précision et f-mesure pour chacune des trois étiquettes choisies ci-dessus, et comparer ces trois scores à ceux des trois classifieurs binaires précédents.
    
5. **Documentation** : livre NLTK, chapitre 2 pour le corpus Reuters, chapitre 6 pour la classification, et http://www.nltk.org/howto/classify.html pour les classifieurs dans NLTK ; Introduction to Information Retrieval (https://nlp.stanford.edu/IR-book/information-retrieval-book.html), chapitre 13, pour une discussion de méthodes de classification, et des exemples de scores obtenus sur certaines étiquettes.

## Définition des fonctions

On va définir des fonctions pour: obtenir le vocabulaire, filtrer, trouver la catégorie adéquate et préparer les données.

In [2]:
import nltk
from nltk.corpus import reuters
from nltk.probability import FreqDist
from nltk.corpus import stopwords
import string

In [3]:
def remove_stop_words(words):
    """
    Remove english stopwords and words of length 1
    inspiration: http://www.nltk.org/book/ch02.html
    words: a list of words
    return words lower case without stopwords
    """
    stopwords = nltk.corpus.stopwords.words('english')
    return [w.lower() for w in words if len(w) > 1 and w.lower() not in stopwords]


def remove_punctuation(words):
    """
    inspiration: https://machinelearningmastery.com/clean-text-machine-learning-python/
    """
    table = str.maketrans('', '', string.punctuation)
    return [w.translate(table) for w in words]


def filter_words(words, rm_punctuation=True, rm_stopwords=True):
    if rm_punctuation:
        words = remove_punctuation(words)
    if rm_stopwords:
        words = remove_stop_words(words)
    
    return words


def get_reuters_vocab(sample_size=500, rm_punctuation=True, rm_stopwords=True):
    words = reuters.words()         
    
    fdist = FreqDist(words)
    
    print('Number of words in reuters: {} total - unique: {} \n'.format(fdist.N(), fdist.B()))
    
    fdist = FreqDist(filter_words(words, rm_punctuation, rm_stopwords) )

    print('After removals: {} total - unique: {} \n'.format(fdist.N(), fdist.B()))

    return [w[0] for w in fdist.most_common(sample_size)]


def vocab_dic(vocab):
    """
    return a vocabulary list as a dictionnary with all values
    set to False
    """
    return {w:False for w in vocab}


reuters_vocabulary_dic = vocab_dic(get_reuters_vocab())

Number of words in reuters: 1720901 total - unique: 41600 

After removals: 964625 total - unique: 30778 



In [13]:
def _category(fileid, wanted_category):
    """
    return a specific category given a fileid. If not found, returns "not_category"
    """
    return wanted_category if wanted_category in reuters.categories(fileid) else f'not_{wanted_category}'


def _words_dic(vocab_dic, words):
    _tmp_dic = dict()
    _tmp_dic.update(vocab_dic)
        
    for w in words:
        if w in vocab_dic:
            _tmp_dic[w] = True
            
    return _tmp_dic
            

def prepare_reuters_data(vocab_dic, wanted_category, rm_punctuation=True, rm_stopwords=True):
    train_data = []
    test_data = []

    for fileid in reuters.fileids():
        category = _category(fileid, wanted_category)
        words = reuters.words(fileid)
        words = filter_words(words, rm_punctuation, rm_stopwords)
        dic = _words_dic(vocab_dic, words)
        if fileid.startswith('test'):
            test_data.append([dic, category])
        else:
            train_data.append([dic, category])
                
    return train_data, test_data

In [5]:
def prepare_pre_cut(train_data, test_data):
    cut_train = int(len(train_data) * 0.8 )
    cut_test = int(len(test_data) * 0.2 )
    return train_data[0:cut_train], test_data[0:cut_test]


reuters_train_data, reuters_test_data = prepare_reuters_data(reuters_vocabulary_dic, 'money-supply')
print(len(reuters_train_data), len(reuters_test_data))

cut_reuters_train_data, cut_reuters_test_data = prepare_pre_cut(reuters_train_data, reuters_test_data)
print(len(cut_reuters_train_data), len(cut_reuters_test_data))

7769 3019
6215 603


In [6]:
from nltk.classify import * 

In [7]:
classifier_2 = nltk.NaiveBayesClassifier.train(cut_reuters_train_data)
print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier_2, cut_reuters_test_data))*100)
classifier_2.show_most_informative_features(10)

Classifier accuracy percent: 93.69817578772802
Most Informative Features
                     cts = True           not_mo : money- =     21.9 : 1.0
                     fed = True           money- : not_mo =     21.8 : 1.0
                      vs = True           not_mo : money- =     20.5 : 1.0
                 company = True           not_mo : money- =     18.7 : 1.0
                    corp = True           not_mo : money- =     18.3 : 1.0
                  supply = True           money- : not_mo =     16.8 : 1.0
                   money = True           money- : not_mo =     15.4 : 1.0
                     000 = True           not_mo : money- =     13.2 : 1.0
                 reserve = True           money- : not_mo =     11.2 : 1.0
              bundesbank = True           money- : not_mo =     10.7 : 1.0


In [8]:
import collections 
from nltk.metrics import precision, recall, f_measure

In [14]:
def obtain_data(wanted_category, cut=True, sampleSize=500, rm_punctuation=True, rm_stopwords=True):
    reuters_vocabulary_dic = vocab_dic(get_reuters_vocab(rm_punctuation, rm_stopwords))
    
    reuters_train_data, reuters_test_data = prepare_reuters_data(reuters_vocabulary_dic, wanted_category)

    if cut:
        reuters_train_data, reuters_test_data = prepare_pre_cut(reuters_train_data, reuters_test_data)
        
    return reuters_train_data, reuters_test_data
    
    
def eval_classifier(trained_classifier, test_data, label):
    """
    Inspiration: https://streamhacker.com/2010/05/17/text-classification-sentiment-analysis-precision-recall/
    """
    ref_set = collections.defaultdict(set)
    test_set = collections.defaultdict(set)
    
    for i, (feats, true_category) in enumerate(test_data):
        ref_set[true_category].add(i)
        observed = trained_classifier.classify(feats)
        test_set[observed].add(i)
    
    print(ref_set, test_set)
    print('Classifier evaluation: ')
    print( 'Precision:', precision(ref_set[label], test_set[label]) )
    print( 'Recall:', recall(ref_set[label], test_set[label]) )
    print( 'F-measure:', f_measure(ref_set[label], test_set[label]) )
    
    
def run_classifier(classifier, wanted_category, cut=True, sampleSize=500, rm_punctuation=True, rm_stopwords=True):
    train_data, test_data = obtain_data(wanted_category, cut, sampleSize, rm_punctuation, rm_stopwords)
    
    print(f'{classifier}')
    
    trained_classifier = classifier.train(train_data)
    
    eval_classifier(trained_classifier, test_data, wanted_category)

    
categories = {
    0: ['money-fx', 'interest', 'money-supply'],
    1: ['grain', 'wheat', 'corn'],
    2: ['crude', 'nat-gas', 'gold']
}

In [15]:
run_classifier(nltk.NaiveBayesClassifier, categories[0][0], cut=True, sampleSize=5000, rm_punctuation=True, rm_stopwords=True)

Number of words in reuters: 1720901 total - unique: 41600 

After removals: 964625 total - unique: 30778 

<class 'nltk.classify.naivebayes.NaiveBayesClassifier'>
defaultdict(<class 'set'>, {'not_money-fx': {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187,